In [1]:
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [2]:
train_datagen = ImageDataGenerator(
        rescale = 1./255,
        rotation_range = 30,
        shear_range = 0.3,
        horizontal_flip = True,
        width_shift_range = 0.1,
        height_shift_range = 0.1,
        zoom_range = 0.25,
)
valid_datagen = ImageDataGenerator(
        rescale = 1./255,
)

In [ ]:
batch_size = 64
img_width = 128
img_height = 128

train_data = train_datagen.flow_from_directory(
    '경로',
    batch_size = batch_size,
    target_size = (img_width, img_height),
    shuffle = True,
)
valid_data = valid_datagen.flow_from_directory(
    '경로',
    target_size = (img_width, img_height),
    batch_size = batch_size,
    shuffle = False,
)

In [ ]:
def visualize_images(images, labels):
    figure, ax = plt.subplots(nrows = 3, ncols = 3, figsize = (12, 14))
    classes = list(train_data.class_indices.keys())
    img_no = 0
    for i in range(3):
        for j in range(3):
            img = images[img_no]
            label_no = np.argmax(labels[img_no])
            
            ax[i,j].imshow(img)
            ax[i,j].set_title(classes[label_no])
            ax[i,j].set_axis_off()
            img_no += 1

images, labels = next(train_data)
visualize_images(images, labels)

In [ ]:
base = MobileNetV2(input_shape = (img_width, img_height, 3), include_top = False,
                  weights = 'imagenet')
base.trainable = True
model = Sequential()
model.add(base)
model.add(GlobalAveragePooling2D())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(6, activation = 'softmax'))
opt = Adam(learning_rate = 0.001)
model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor = 'val_accuracy', patience = 1, verbose = 1)
early_stop = EarlyStopping(monitor = 'val_accuracy', patience = 5, verbose = 1,
                          resotre_best_weights = True)
check_point = ModelCheckpoint('best_model.h5', monitor = 'val_accuracy', verbose = 1,
                             save_best_only = True)

In [ ]:
history = model.fit(train_data, epochs = 50, validation_data = valid_data,
                   callbacks = [early_stop, reduce_lr, check_point])